In [1]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
from random import sample
import os

import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable 

сделать матрицу

In [2]:
os.listdir('data')    

['Dewas_NDVI.csv',
 'Dewas_points.csv',
 'Kaithal_NDVI.csv',
 'Kaithal_points.csv',
 'Karnal_NDVI.csv',
 'Karnal_points.csv']

In [3]:
data_points = pd.read_csv('data/Karnal_points.csv')

In [4]:
data_ndvi = pd.read_csv('data/Karnal_NDVI.csv')

In [5]:
data = data_points.merge(data_ndvi, left_on='gfid', right_on='gfid')

In [6]:
data['month'] = data['date'].apply(pd.to_datetime).dt.month
data['day'] = data['date'].apply(pd.to_datetime).dt.day

In [7]:
data_ndvi.head()

,gfid,datenum,date,ndvi
0,54001,0,2020-10-20,0.737
1,54001,1,2020-10-21,0.727
2,54001,2,2020-10-22,0.714
3,54001,3,2020-10-23,0.697
4,54001,4,2020-10-24,0.676


In [8]:
i = 1
ids_not_full = []
for id_ in data_points['gfid'].values:
    shp = data_ndvi[data_ndvi['gfid'] == id_].shape[0]
    if shp <= 160:
        #print(id_, ' ', shp)
        i += 1
        ids_not_full += [id_]
#print(i)

In [9]:
data2 = data[~data['gfid'].isin(ids_not_full)]

In [10]:
gfids = data2['gfid'].unique()

In [11]:
from numpy.random import choice

In [12]:
import numpy as np

In [13]:
np.random.seed(42)

In [14]:
np.random.shuffle(gfids)

In [15]:
train_ids = gfids[:210]
test_ids = gfids[210:]

In [16]:
indices = {}
indices['train'] = list([int(x) for x in train_ids])
indices['test'] = list([int(x) for x in test_ids])

import json
with open('Karnal.json', 'w') as f:
    json.dump(indices, f)

In [17]:
data2.head()

,gfid,state,district,village,lon,lat,wheat,datenum,date,ndvi,month,day
0,54001,Haryana,Karnal,Kabulpur Khera,76.707176,29.55072,1,0,2020-10-20,0.737,10,20
1,54001,Haryana,Karnal,Kabulpur Khera,76.707176,29.55072,1,1,2020-10-21,0.727,10,21
2,54001,Haryana,Karnal,Kabulpur Khera,76.707176,29.55072,1,2,2020-10-22,0.714,10,22
3,54001,Haryana,Karnal,Kabulpur Khera,76.707176,29.55072,1,3,2020-10-23,0.697,10,23
4,54001,Haryana,Karnal,Kabulpur Khera,76.707176,29.55072,1,4,2020-10-24,0.676,10,24


In [18]:
data2 = data2.dropna()

In [19]:
data2.shape

(53505, 12)

In [20]:
data2.shape

(53505, 12)

for i in (3, 5, 7, 9):
    data2[f'ndvi_{i}'] = data2['ndvi'].rolling(i).mean()

In [21]:
datasets_train = []
datasets_test = []
y_train = []
y_test = []
for _ in gfids:
    rolling_data = []
    rolling_data.append(data2[data2['gfid']==_]['ndvi'].iloc[4:])
    for i in range(2, 11):
        rolling_data.append(data2[data2['gfid']==_]['ndvi'].rolling(i).mean())
    data_1_rolling = pd.concat(rolling_data, axis=1)
    data_1_rolling = data_1_rolling.dropna()
    data_1_rolling.columns = ['ndvi'] + [f'ndvi_{i}' for i in range(2,11)]
    if _ in train_ids:
        datasets_train.append(data_1_rolling[:150].values.reshape(15,10,10))
        y_train += [data_points.loc[data_points['gfid'] == _, 'wheat'].iloc[0]]
    else:
        datasets_test.append(data_1_rolling[:150].values.reshape(15,10,10))
        y_test += [data_points.loc[data_points['gfid'] == _, 'wheat'].iloc[0]]

In [22]:
data_train = torch.stack([torch.Tensor(dataset) for dataset in datasets_train])
data_test = torch.stack([torch.Tensor(dataset) for dataset in datasets_test])

In [23]:
def calc_out(h_in, w_in, ker, pad=0, stride=1):
    h_out = (h_in + 2 * pad - (ker - 1) - 1) / (stride) + 1
    w_out = (w_in + 2 * pad - (ker - 1) - 1) / (stride) + 1
    return h_out, w_out

In [24]:
class MyCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes  #number of classes
        
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=15, out_channels=30, kernel_size=2, stride=2, padding=1),  # (b x 30 x 6 x 6)
            nn.ReLU(),
            nn.Conv2d(30, 64, 2, padding=0, stride=1),  # (b x 64 x 5 x 5)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=0),  # (b x 64 x 4 x 4)
        )
        # classifier is just a name for linear layers
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.1, inplace=True),
            nn.Linear(in_features=(64 * 4 * 4), out_features=256),
            nn.ReLU(),
            #nn.Dropout(p=0.5, inplace=True),
            #nn.Linear(in_features=256, out_features=256),
            #nn.ReLU(),
            nn.Linear(in_features=256, out_features=num_classes),
            nn.Sigmoid()
        )
        #self.init_bias()

    def init_bias(self):
        for layer in self.net:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean=0, std=0.01)
                nn.init.constant_(layer.bias, 0)
        # original paper = 1 for Conv2d layers 2nd, 4th, and 5th conv layers
        nn.init.constant_(self.net[0].bias, 1)
        nn.init.constant_(self.net[2].bias, 1)
        #nn.init.constant_(self.net[4].bias, 1)

    def forward(self, x):
        """
        Pass the input through the net.
        Args:
            x (Tensor): input tensor
        Returns:
            output (Tensor): output tensor
        """
        x = self.net(x)
        x = x.view(-1, 64 * 4 * 4)  # reduce the dimensions for linear layer input
        return self.classifier(x)

In [25]:
MyCNN = MyCNN(num_classes=2)

In [26]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(params=MyCNN.parameters(), lr=0.0001)#1, weight_decay=0.5)

In [27]:
torch.autograd.set_detect_anomaly(True)

In [28]:
y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

y_train_tensors = y_train_tensors.type(torch.LongTensor)
y_test_tensors = y_test_tensors.type(torch.LongTensor)

In [29]:
X = data_train
y = y_train_tensors

In [30]:
res = []
n_epochs = 100  # or whatever
batch_size = 30  # or whatever

for epoch in range(n_epochs):

    # X is a torch Variable
    permutation = torch.randperm(X.size()[0])

    for i in range(0, X.size()[0], batch_size):
        optimizer.zero_grad()

        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X[indices], y[indices]

        outputs = MyCNN.forward(batch_x) #forward pass

        # obtain the loss function
        loss = criterion(outputs, batch_y)
        res.append(loss)
        loss.backward() #calculates the loss of the loss function

        optimizer.step() #improve from loss, i.e backprop
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 0.68721
Epoch: 1, loss: 0.67702
Epoch: 2, loss: 0.69775
Epoch: 3, loss: 0.67789
Epoch: 4, loss: 0.70636
Epoch: 5, loss: 0.67549
Epoch: 6, loss: 0.63633
Epoch: 7, loss: 0.69148
Epoch: 8, loss: 0.68334
Epoch: 9, loss: 0.68927
Epoch: 10, loss: 0.68439
Epoch: 11, loss: 0.64875
Epoch: 12, loss: 0.62527
Epoch: 13, loss: 0.64275
Epoch: 14, loss: 0.64285
Epoch: 15, loss: 0.70570
Epoch: 16, loss: 0.61031
Epoch: 17, loss: 0.63051
Epoch: 18, loss: 0.59401
Epoch: 19, loss: 0.65769
Epoch: 20, loss: 0.63224
Epoch: 21, loss: 0.54587
Epoch: 22, loss: 0.62619
Epoch: 23, loss: 0.62445
Epoch: 24, loss: 0.57957
Epoch: 25, loss: 0.62494
Epoch: 26, loss: 0.64481
Epoch: 27, loss: 0.58387
Epoch: 28, loss: 0.73108
Epoch: 29, loss: 0.60892
Epoch: 30, loss: 0.66149
Epoch: 31, loss: 0.60979
Epoch: 32, loss: 0.62212
Epoch: 33, loss: 0.48903
Epoch: 34, loss: 0.64950
Epoch: 35, loss: 0.69438
Epoch: 36, loss: 0.75232
Epoch: 37, loss: 0.51765
Epoch: 38, loss: 0.58584
Epoch: 39, loss: 0.53924
Epoch: 40,

In [31]:
X_test_tensors = data_test

In [32]:
test_predict = MyCNN(X_test_tensors)#forward pass 

In [33]:
test_predict

tensor([[0.9230, 0.1103],
        [0.0640, 0.9389],
        [0.9211, 0.1085],
        [0.0169, 0.9835],
        [0.0163, 0.9841],
        [0.0924, 0.9147],
        [0.8167, 0.2093],
        [0.0464, 0.9567],
        [0.0403, 0.9626],
        [0.0291, 0.9727],
        [0.9649, 0.0555],
        [0.2218, 0.7918],
        [0.6880, 0.3446],
        [0.1787, 0.8345],
        [0.6320, 0.3970],
        [0.0437, 0.9583],
        [0.0296, 0.9717],
        [0.7948, 0.2450],
        [0.9615, 0.0596],
        [0.7996, 0.2346],
        [0.1352, 0.8762],
        [0.9467, 0.0786],
        [0.1973, 0.8166],
        [0.6745, 0.3548],
        [0.0180, 0.9833],
        [0.1066, 0.8972],
        [0.0167, 0.9837],
        [0.0460, 0.9538],
        [0.8355, 0.2117],
        [0.4634, 0.5609],
        [0.0074, 0.9931],
        [0.2137, 0.8059],
        [0.1906, 0.8329],
        [0.9395, 0.0860],
        [0.0245, 0.9772],
        [0.0557, 0.9466],
        [0.9363, 0.0813],
        [0.0217, 0.9794],
        [0.0

In [34]:
test_predict.argmax(dim=1)

tensor([0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
        1, 0, 1, 1, 1, 1, 1, 0, 0, 1])

In [35]:
# 100 эпох, батч 30
print(classification_report(y_test_tensors, torch.argmax(test_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.64      0.70      0.67        20
           1       0.83      0.79      0.81        38

    accuracy                           0.76        58
   macro avg       0.73      0.74      0.74        58
weighted avg       0.77      0.76      0.76        58



In [36]:
accuracy_score(y_test_tensors, torch.argmax(test_predict, dim=1))

0.7586206896551724

In [37]:
roc_auc_score(y_test_tensors, torch.argmax(test_predict, dim=1))

0.7447368421052631

In [38]:
train_predict = MyCNN(X)#forward pass 

In [39]:
# 100 эпох, батч 30
print(classification_report(y_train_tensors, torch.argmax(train_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.68      0.67      0.67        88
           1       0.76      0.77      0.77       122

    accuracy                           0.73       210
   macro avg       0.72      0.72      0.72       210
weighted avg       0.73      0.73      0.73       210



In [40]:
accuracy_score(y_train_tensors, torch.argmax(train_predict, dim=1))

0.7285714285714285

In [41]:
roc_auc_score(y_train_tensors, torch.argmax(train_predict, dim=1))

0.720473174366617